In [152]:
import pandas as pd
from time import time
import nba_api
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import playerawards

In [153]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [154]:
# function which collects player seasons and drops before modern 3 point era and insufficient data

def collect_seasons(id):
    season_filter = 1998 # the first year of the modern 3 point era

    career = playercareerstats.PlayerCareerStats(player_id=id)
    df = career.get_data_frames()[0]

    df['SEASON_ID'] = df['SEASON_ID'].apply(lambda x: int(x[:4]) + 1) #make seasons numeric type
    df = df[df['SEASON_ID'] >= season_filter] # drop any seasons before modern era

    return df

In [155]:
# function which gets player awards

# fun fact and test case: dominique wilkins in 94 made an All-NBA team
# but played for two teams that year

def collect_awards(id):
    awards = playerawards.PlayerAwards(player_id=id)

    df = awards.get_data_frames()[0]

    df = df[df['DESCRIPTION'] == 'All-NBA']
    df = df[['SEASON', 'ALL_NBA_TEAM_NUMBER']]

    df['SEASON'] = df['SEASON'].apply(lambda x: int(x[:4]) + 1) #make seasons numeric type

    return df

In [156]:
# function which joins the awards and seasons

def stat_join(seasons, awards):
    df = seasons.join(awards.set_index('SEASON'), on='SEASON_ID')
    df['ALL_NBA_TEAM_NUMBER'] = df['ALL_NBA_TEAM_NUMBER'].fillna(0)
    return df

In [159]:
# function which loops through each player

nba_players = players.get_players()

final_df = pd.DataFrame()

for count, player in enumerate(players.get_players()[:100]):
    id = player['id']
    name = player['full_name']

    start = time()
    print(f"Processing {name}...")

    seasons = collect_seasons(id)
    awards = collect_awards(id)
    df = stat_join(seasons, awards)
    df['NAME'] = player['full_name']

    if count == 0:
        final_df = df
    else:
        final_df = pd.concat([final_df, df])
    
    end = time()
    print(f"Took {end - start} seconds to finish")

Processing Alaa Abdelnaby...
Took 0.43891334533691406 seconds to finish
Processing Zaid Abdul-Aziz...
Took 0.42555761337280273 seconds to finish
Processing Kareem Abdul-Jabbar...
Took 0.31864452362060547 seconds to finish
Processing Mahmoud Abdul-Rauf...


/tmp/ipykernel_7207/1316389018.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, df])


Took 0.4627072811126709 seconds to finish
Processing Tariq Abdul-Wahad...
Took 0.40835070610046387 seconds to finish
Processing Shareef Abdur-Rahim...
Took 1.373192310333252 seconds to finish
Processing Tom Abernethy...
Took 1.6619038581848145 seconds to finish
Processing Forest Able...


/tmp/ipykernel_7207/1316389018.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, df])


Took 1.3359005451202393 seconds to finish
Processing John Abramovic...


...

Took 2.419771909713745 seconds to finish
Processing Derek Anderson...
Took 2.4767277240753174 seconds to finish
Processing Dwight Anderson...
Took 2.506911039352417 seconds to finish
Processing Eric Anderson...
Took 1.5328259468078613 seconds to finish
Processing Greg Anderson...
Took 0.4900016784667969 seconds to finish
Processing James Anderson...
Took 1.814366102218628 seconds to finish
Processing Jerome Anderson...


/tmp/ipykernel_7207/1316389018.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, df])


Took 2.625643730163574 seconds to finish
Processing Justin Anderson...
Took 2.548380136489868 seconds to finish
Processing Kenny Anderson...
Took 1.7660069465637207 seconds to finish
Processing Kim Anderson...


/tmp/ipykernel_7207/1316389018.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, df])


Took 3.376744031906128 seconds to finish
Processing Kyle Anderson...
Took 0.6476292610168457 seconds to finish
Processing Michael Anderson...
Took 2.305745840072632 seconds to finish
Processing Mitchell Anderson...
Took 1.6038563251495361 seconds to finish
Processing Nick Anderson...
Took 2.5454187393188477 seconds to finish


In [160]:
final_df

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,ALL_NBA_TEAM_NUMBER,NAME
7,51,1998,00,1610612758,SAC,29.0,31,0,530.0,103,273,0.377,5,31,0.161,16,16,1.000,6.0,31.0,37.0,58,16.0,1.0,19.0,31,227,0,Mahmoud Abdul-Rauf
8,51,2001,00,1610612763,VAN,32.0,41,0,485.0,120,246,0.488,4,14,0.286,22,29,0.759,5.0,20.0,25.0,76,9.0,1.0,26.0,50,266,0,Mahmoud Abdul-Rauf
0,1505,1998,00,1610612758,SAC,23.0,59,16,959.0,144,357,0.403,4,19,0.211,84,125,0.672,44.0,72.0,116.0,51,35.0,13.0,65.0,81,376,0,Tariq Abdul-Wahad
1,1505,1999,00,1610612758,SAC,24.0,49,49,1205.0,177,407,0.435,6,21,0.286,94,136,0.691,72.0,114.0,186.0,50,50.0,16.0,70.0,121,454,0,Tariq Abdul-Wahad
2,1505,2000,00,1610612753,ORL,25.0,46,46,1205.0,223,515,0.433,2,21,0.095,115,151,0.762,77.0,162.0,239.0,72,53.0,16.0,87.0,116,563,0,Tariq Abdul-Wahad
3,1505,2000,00,1610612743,DEN,25.0,15,10,373.0,51,131,0.389,1,2,0.500,31,42,0.738,24.0,28.0,52.0,26,6.0,12.0,19.0,31,134,0,Tariq Abdul-Wahad
4,1505,2000,00,0,TOT,25.0,61,56,1578.0,274,646,0.424,3,23,0.130,146,193,0.756,101.0,190.0,291.0,98,59.0,28.0,106.0,147,697,0,Tariq Abdul-Wahad
5,1505,2001,00,1610612743,DEN,26.0,29,12,421.0,43,111,0.387,4,10,0.400,21,36,0.583,14.0,45.0,59.0,22,14.0,13.0,34.0,54,111,0,Tariq Abdul-Wahad
6,1505,2002,00,1610612743,DEN,27.0,20,12,419.0,55,145,0.379,1,2,0.500,24,32,0.750,39.0,39.0,78.0,22,18.0,9.0,24.0,51,135,0,Tariq Abdul-Wahad
7,1505,2002,00,1610612742,DAL,27.0,4,0,24.0,0,2,0.000,0,0,0.000,0,1,0.000,2.0,4.0,6.0,2,2.0,1.0,3.0,5,0,0,Tariq Abdul-Wahad


In [161]:
final_df.to_csv('../data/data.csv')